In [ ]:
!git clone https://github.com/spMohanty/PlantVillage-Dataset.git

Cloning into 'PlantVillage-Dataset'...
remote: Enumerating objects: 163229, done.
remote: Total 163229 (delta 0), reused 0 (delta 0), pack-reused 163229
Receiving objects: 100% (163229/163229), 2.00 GiB | 25.93 MiB/s, done.
Resolving deltas: 100% (99/99), done.
Updating files: 100% (182401/182401), done.


In [ ]:
!pip install intel-tensorflow[cpu]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.7/244.7 MB 3.1 MB/s eta 0:00:00


In [ ]:
import os
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split

In [ ]:
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'
os.environ['TF_ENABLE_MKL_OPTS'] = '1'

In [ ]:
dataset_dir = '/content/PlantVillage-Dataset/raw/segmented'

In [ ]:
image_size = (224, 224)
batch_size = 32
random.seed(42)
train_dir = 'path/to/train'
test_dir = 'path/to/test'
train_ratio = 0.8

In [ ]:
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [ ]:
for class_folder in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_folder)
    if os.path.isdir(class_dir):
        image_files = os.listdir(class_dir)
        train_files, test_files = train_test_split(image_files, train_size=train_ratio, random_state=42)
        train_class_dir = os.path.join(train_dir, class_folder)
        os.makedirs(train_class_dir, exist_ok=True)
        for file in train_files:
            src_path = os.path.join(class_dir, file)
            dst_path = os.path.join(train_class_dir, file)
            shutil.copy(src_path, dst_path)

        test_class_dir = os.path.join(test_dir, class_folder)
        os.makedirs(test_class_dir, exist_ok=True)
        for file in test_files:
            src_path = os.path.join(class_dir, file)
            dst_path = os.path.join(test_class_dir, file)
            shutil.copy(src_path, dst_path)

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(train_dir,
                                              target_size=image_size,
                                              batch_size=batch_size,
                                              class_mode='binary')
test_generator = datagen.flow_from_directory(test_dir,
                                             target_size=image_size,
                                             batch_size=batch_size,
                                             class_mode='binary')

Found 43430 images belonging to 38 classes.
Found 10876 images belonging to 38 classes.


In [ ]:
model = Sequential()
model.add(Flatten(input_shape=(image_size[0], image_size[1], 3)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_generator, epochs=3, validation_data=test_generator)

Epoch 1/3
1358/1358 [==============================] - 418s 307ms/step - loss: -30738224.0000 - accuracy: 0.0114 - val_loss: -88887048.0000 - val_accuracy: 0.0115
Epoch 2/3
1358/1358 [==============================] - 383s 282ms/step - loss: -193476320.0000 - accuracy: 0.0114 - val_loss: -315680544.0000 - val_accuracy: 0.0115
Epoch 3/3
1358/1358 [==============================] - 403s 297ms/step - loss: -473727360.0000 - accuracy: 0.0114 - val_loss: -643674944.0000 - val_accuracy: 0.0115


In [ ]:
model.save('plant_disease_model.h5')

In [ ]:
loaded_model = tf.keras.models.load_model('plant_disease_model.h5')

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
custom_image_path = '/content/download.jpg'
custom_image = image.load_img(custom_image_path, target_size=image_size)
custom_image = image.img_to_array(custom_image)
custom_image = np.expand_dims(custom_image, axis=0)
custom_image = custom_image / 255.0

In [ ]:
predictions = loaded_model.predict(custom_image)
class_index = predictions.argmax()
class_label = train_generator.class_indices
for key, value in class_label.items():
    if value == class_index:
        class_name = key
        break

1/1 [==============================] - 0s 113ms/step


In [ ]:
print(class_label)

{'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Blueberry___healthy': 4, 'Cherry_(including_sour)___Powdery_mildew': 5, 'Cherry_(including_sour)___healthy': 6, 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7, 'Corn_(maize)___Common_rust_': 8, 'Corn_(maize)___Northern_Leaf_Blight': 9, 'Corn_(maize)___healthy': 10, 'Grape___Black_rot': 11, 'Grape___Esca_(Black_Measles)': 12, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 13, 'Grape___healthy': 14, 'Orange___Haunglongbing_(Citrus_greening)': 15, 'Peach___Bacterial_spot': 16, 'Peach___healthy': 17, 'Pepper,_bell___Bacterial_spot': 18, 'Pepper,_bell___healthy': 19, 'Potato___Early_blight': 20, 'Potato___Late_blight': 21, 'Potato___healthy': 22, 'Raspberry___healthy': 23, 'Soybean___healthy': 24, 'Squash___Powdery_mildew': 25, 'Strawberry___Leaf_scorch': 26, 'Strawberry___healthy': 27, 'Tomato___Bacterial_spot': 28, 'Tomato___Early_blight': 29, 'Tomato___Late_blight': 30, 'Tomato

In [ ]:
print('Predicted Class:', class_name)

Predicted Class: Apple___Apple_scab
